In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import random
from tqdm import tqdm

In [18]:
import lxml.etree

In [ ]:
pages = [str(i) for i in range(0, 272, 1)] # page iterations

In [ ]:
page_urls = []
for page in pages:
    base_url = 'https://www.ted.com/talks?page='+page
    page_urls.append(base_url)

In [5]:
page_url = 'https://www.ted.com/talks?page=1'

In [30]:
talks = [] # also called add_links
page = requests.get(page_url)
soup = BeautifulSoup(page.content, 'html.parser')

# find href links to talks in page content under <a> tags
pbar = tqdm(total=len(page_urls), dynamic_ncols=True, colour= 'green')
for i, page in enumerate(page_urls):
    for link in soup.find_all('a'):
        time.sleep(0.5)
        pbar.update(1)
        pbar.set_description(f'Downloading page {i+1}/{len(page_urls)}', refresh=True)

        if link.has_attr('href') and link['href'].startswith('/talks/'):
            ted_url = 'https://www.ted.com'
            #check if link already exists in talks list
            if ted_url + link['href'] not in talks:
                #concat 'https://www.ted.com' + link['href'] to talks list
                talks.append(ted_url + link['href'])
            else:
        # if link already exists in talks list, skip it
                continue
            time.sleep(0.5)
pbar.close()

['https://www.ted.com/talks/susanne_buckley_zistel_what_caused_the_rwandan_genocide', 'https://www.ted.com/talks/conor_russomanno_a_powerful_new_neurotech_tool_for_augmenting_your_mind', 'https://www.ted.com/talks/peter_singer_a_modern_argument_for_the_rights_of_animals', 'https://www.ted.com/talks/sahar_zand_why_iranians_are_cutting_their_hair_for_woman_life_freedom', 'https://www.ted.com/talks/shannon_odell_are_solar_panels_worth_it', 'https://www.ted.com/talks/angus_hervey_why_are_we_so_bad_at_reporting_good_news', 'https://www.ted.com/talks/sheryl_lee_ralph_a_3_step_guide_to_believing_in_yourself', 'https://www.ted.com/talks/carolyn_harris_why_is_marie_antoinette_so_controversial', 'https://www.ted.com/talks/rainn_wilson_kung_fu_star_trek_and_the_many_paths_to_spirituality', 'https://www.ted.com/talks/richard_reeves_how_to_solve_the_education_crisis_for_boys_and_men', 'https://www.ted.com/talks/mena_fombo_no_you_cannot_touch_my_hair_jun_2023', 'https://www.ted.com/talks/iseult_gill

In [120]:
talks[5]

'https://www.ted.com/talks/angus_hervey_why_are_we_so_bad_at_reporting_good_news'

In [138]:

# find views of talk under <div class="text-sm w-full truncate text-gray-900" data-testid="talk-meta">
response = requests.get(talks[5])
soup = BeautifulSoup(response.text, 'lxml')
views_schema = soup.find_all('div', class_='text-sm w-full truncate text-gray-900')
views = views_schema[0].get_text().strip().split()[0]

print (views)


407,040


In [ ]:
pbar = tqdm(total=len(talks), dynamic_ncols=True, colour= 'green')

for i, ad in enumerate(talks):
    #-------create dataframe--------#
    df = pd.DataFrame(columns=["author", "talk", "description", "likes", "views"])

    time.sleep(2)
    pbar.update(1)
    response = requests.get(talks[5])
    soup = BeautifulSoup(response.text, 'lxml')


    #--------Title Schema------------#
    title_schema = soup.find('head').find('title').text.strip()

    #--------Description Schema------------#
    description_schema = soup.find('head').find('meta', attrs={'name':'description'})['content'].strip()

    #--------Likes Schema------------#
    likes_schema = soup.find_all('span')[0].get_text().strip()

    #--------Views Schema------------#
    views_schema = soup.find_all('div', class_='text-sm w-full truncate text-gray-900')


    # get author name from title 
    author = title_schema.split(':')[0]
    talk = title_schema.split(':')[1].strip().replace('| TED Talk', '')
    description = description_schema
    likes = likes_schema.replace('(', '').replace(')', '')
    try: 
        views = views_schema[0].get_text().strip().split()[0]
    except:
        views = 0

    pbar.set_description(f'Downloading {talk}', refresh=True)

    # add to dataframe
    df = df.append({'author': author, 'talk': talk, 'description': description, 'likes': likes, 'views': views}, ignore_index=True)

 # ----------------------------------------Saving to Database--------------------------------------------#
 
    conn = sqlite3.connect('talks.db')
    cur = conn.cursor()
                
    for i in range(len(df)):
        cur.execute("INSERT INTO talks (author, talk, description, likes, views) VALUES (?, ?, ?, ?, ?)", (df.iloc[i]['author'], df.iloc[i]['talk'], df.iloc[i]['description'], df.iloc[i]['likes'], df.iloc[i]['views']))
    conn.commit()

    time.sleep(1)
    pbar.set_description(f'Adding {talk} to database', refresh=True)
    pbar.update(1)

    time.sleep(1)
    cur.execute("SELECT * FROM talks")
    rows = cur.fetchall()

    pbar.set_description(f"There are {len(rows)} records in the database", refresh=True)
    conn.close()
    time.sleep(1)
pbar.close()

